## Bike and Dock Availability

In [6]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from datetime import timedelta
from matplotlib.dates import DateFormatter

In [7]:
# df = pd.read_parquet('~/bikeshare/tripdata/202009-citibike-tripdata.parquet',\
#                     columns=['starttime','start station id','stoptime','end station id'])
# df.info()

#### Read Trip Data
Read September 2020 tripdata file into dataframe

In [8]:
df = pd.read_csv('../../raw/202003-citibike-tripdata.csv',\
                 usecols=['starttime','start station id','stoptime','end station id'],\
                 parse_dates=['starttime','stoptime'])

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1068457 entries, 0 to 1068456
Data columns (total 4 columns):
 #   Column            Non-Null Count    Dtype         
---  ------            --------------    -----         
 0   starttime         1068457 non-null  datetime64[ns]
 1   stoptime          1068457 non-null  datetime64[ns]
 2   start station id  1068457 non-null  int64         
 3   end station id    1068457 non-null  int64         
dtypes: datetime64[ns](2), int64(2)
memory usage: 32.6 MB


#### Reblancing 
Read file with rebalancing movements.   
Note: requires pyarrow, from command line `conda install -c conda-forge pyarrow`

In [10]:
dfrebal=pd.read_parquet('../bikeshare/202009-citibike-reblance.parquet')

ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.

In [ ]:
dfrebal.info()

#### Combine
Concatenate actual trips with rebalancing movements

In [ ]:
df = pd.concat([df,dfrebal])
df.reset_index(drop=True, inplace=True)
df

#### Restructure 

Create `dfs` with `startime`, `start station id` and `act` as -1 to indicate a bike pick-up

In [ ]:
dfs=df[['starttime','start station id']].assign(act=-1)

Create `dfe` with `stoptime`, `end station id` and `act` as +1 to indicate a bike drop-off

In [ ]:
dfe=df[['stoptime','end station id']].assign(act=1)

Rename the columns to be the same in both dataframes.
Then create new dataframe `dfse` by concatentating start and end rows.  

In [ ]:
dfs.columns=['docktime','stationid','act']
dfe.columns=['docktime','stationid','act']
dfse=pd.concat([dfs,dfe])

### Sort by Time
Finally sort combined start/end rows all by `docktime`

In [ ]:
dfse.sort_values(by=['docktime'], inplace=True) 
dfse.reset_index(drop=True, inplace=True) 
dfse.head(100)

### Get Station Names

In [ ]:
dfstations = \
  pd.read_csv('~/bikeshare/202009-citibike-tripdata.csv',\
  usecols=['start station id','start station name']).\
  drop_duplicates()                
dfstations.columns=['stationid','station name']
dfstations.set_index('stationid' ,  inplace=True)

In [ ]:
dfstations

### One Station 
Create datafame for all bikes in/out to one station

Get the station id for one station

In [ ]:
station = 'W 31 St & 7 Ave'
sid = dfstations.loc[dfstations['station name']==station].index[0]

Create new dataframe `dfstation` with rows for one station

In [ ]:
dfstation = dfse.loc[(dfse.stationid==sid) ]
dfstation.reset_index(drop=True, inplace=True)

Calculate running sum of `act` as `cnt`.  Note that it starts counting at zero, so eventually there will be a negative count. 

In [ ]:
dfstation = dfstation.assign(cnt = dfstation.act.cumsum())
dfstation.head(10)

#### Determine likely starting number of bikes for this station 
Calculate lowest value for a running sum of `act` which will be a negative number.  
Add its absolute value to first value for `act` to get starting bike count 

In [ ]:
dfstation.at[0, 'act'] =+ abs(dfstation.act.cumsum().min()) 

In [ ]:
dfstation = dfstation.assign(cnt = dfstation.act.cumsum())
dfstation.head(10)

Plot availabilty for the month

In [ ]:
sns.set(font_scale = 1)
plt.figure(figsize=(20,8))
ax=sns.lineplot(data=dfstation , x='docktime', y='cnt' ) 
ax.set_xlabel('Day') 
ax.set_ylabel('Available Bikes') ;

#### Plot availabilty for one day
Using Sept 8 (day after labor day)

In [ ]:
sns.set(font_scale = 2)
plt.figure(figsize=(20,8))
ax=sns.lineplot(data=dfstation.loc[dfstation.docktime.dt.day == 8] , x='docktime', y='cnt' ) 
ax.set_ylabel('Available Bikes')
ax.set_xlabel('Hour of Day') 
ax.set_title('Citi Bike Station - ' + station +' - September 8, 2020')
ax.xaxis.set_major_formatter(DateFormatter("%H")) ;

#### Create function to plot one station 

In [ ]:
def availability (station,day):
    # inputs: station name, day 
    # requires: dfstations, dfse 
    sid = dfstations.loc[dfstations['station name']==station].index[0] # lookup station id 
    dfstation = dfse.loc[(dfse.stationid==sid) ] # create dataframe for a specified station 
    dfstation.reset_index(drop=True, inplace=True)
    dfstation = dfstation.assign(cnt = dfstation.act.cumsum()) # get running total 
    dfstation.at[0, 'act'] =+ abs(dfstation.act.cumsum().min()) # find sub-zero bike count
    dfstation = dfstation.assign(cnt = dfstation.act.cumsum()) # recalculate running total 
    
    # Create chart 
    sns.set(font_scale = 2)
    plt.figure(figsize=(20,8))
    ax=sns.lineplot(data=dfstation.loc[dfstation.docktime.dt.day == day] , x='docktime', y='cnt' ) 
    ax.set_ylabel('Available Bikes')
    ax.set_xlabel('Hour of Day') 
    ax.set_title('Citi Bike Station - ' + station +' - September ' + str(day))
    ax.xaxis.set_major_formatter(DateFormatter("%H")) ;
    

Get list of all station names

In [ ]:
list(dfstations['station name'])

In [ ]:
availability ('W 52 St & 6 Ave',8)

In [ ]:
availability ('1 Ave & E 18 St', 9)